# 8. OpenAI Responses Target

In this demo, we show an example of the `OpenAIResponseTarget`. [Responses](https://platform.openai.com/docs/api-reference/responses) is a newer protocol than chat completions and provides additional functionality with a somewhat modified API. The allowed input types include text, image, web search, file search, functions, reasoning, and computer use.

Before you begin, ensure you are set up with the correct version of PyRIT installed and have secrets configured as described [here](../../setup/populating_secrets.md).
## OpenAI Configuration

Like most targets, all `OpenAITarget`s need an `endpoint` and often also needs a `model` and a `key`. These can be passed into the constructor or configured with environment variables (or in .env).

- endpoint: The API endpoint (`OPENAI_RESPONSES_ENDPOINT` environment variable). For OpenAI, these are just "https://api.openai.com/v1/responses".
- auth: The API key for authentication (`OPENAI_RESPONSES_KEY` environment variable).
- model_name: The model to use (`OPENAI_RESPONSES_MODEL` environment variable). For OpenAI, these are any available model name and are listed here: "https://platform.openai.com/docs/models".

In [ ]:
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_target import OpenAIResponseTarget

initialize_pyrit(memory_db_type=IN_MEMORY)

target = OpenAIResponseTarget()

orchestrator = PromptSendingOrchestrator(objective_target=target)

response = await orchestrator.run_attack_async(objective="Tell me a joke")  # type: ignore
await response.print_conversation_async()  # type: ignore

In [1]:
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target.openai.openai_response_target import OpenAIResponseTarget

initialize_pyrit(memory_db_type=IN_MEMORY)


async def get_current_weather(args):
    # args will be like: {"location": "Boston", "unit": "celsius"}
    return {"weather": "Sunny", "temp_c": 22, "location": args["location"], "unit": args["unit"]}


function_tool = {
    "type": "function",
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. Boston, MA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location", "unit"],
    },
}

target = OpenAIResponseTarget(
    py_functions={"get_current_weather": get_current_weather},
    extra_body_parameters={
        "tools": [function_tool],
        "tool_choice": "auto"
    }
)

prompt_piece = PromptRequestPiece(
    role="user",
    original_value="What is the weather in Boston in celsius? Use the get_current_weather function.",
    original_value_data_type="text"
)
prompt_request = PromptRequestResponse(
    request_pieces=[prompt_piece]
)

response = await target.send_prompt_async(prompt_request=prompt_request)

for idx, piece in enumerate(response.request_pieces):
    print(f"{idx} | {piece.role}: {piece.original_value}")

0 | assistant: The current weather in Boston is sunny with a temperature of 22°C.
